## Homework

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.


## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19




In [1]:
import pandas as pd
import sklearn as sk
import numpy as np
from datetime import timedelta
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error

In [2]:
dfj = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
dff = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [3]:
dfj.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [4]:
num_cols = len(dfj.columns)
print(num_cols)

19


> Number of columns: 19


## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

* 32.59
* 42.59
* 52.59
* 62.59



In [5]:
dfj['duration'] = (dfj.tpep_dropoff_datetime - dfj.tpep_pickup_datetime).dt.total_seconds() / 60
standard_deviation = dfj.duration.std()
print(standard_deviation)

42.59435124195458


> Standard deviation: 42.59435124195458


## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* 98%




In [6]:
# Filter records
dfj_filt = dfj[(dfj['duration'] >= 1) & (dfj['duration'] <= 60) ]

frac_records_left = len(dfj_filt)/len(dfj)
print(f"Fraction records left: {frac_records_left}")

Fraction records left: 0.9812202822125979


> Fraction records left: 0.9812202822125979

## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515
* 715


In [7]:
features = ['PULocationID', 'DOLocationID']

# Convert the catergoricals to strings (they are integers originally).
dicts = dfj_filt[features].astype(str).to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(dicts)
rows, cols = X_train.shape

print(f"The dimensionality of the feature matrix is: {cols}")



The dimensionality of the feature matrix is: 515


> The dimensionality of the feature matrix is: 515



## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters, where duration is the response variable
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 3.64
* 7.64
* 11.64
* 16.64



In [8]:
# Get the duration in seconds
y_train = dfj_filt['duration'].values

# Train
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [9]:
y_pred = lr.predict(X_train)

mse = mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mse)

print(f"RMSE on train is is {rmse}")

RMSE is 7.6492624397080675


> RMSE is 7.6492624397080675


## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

* 3.81
* 7.81
* 11.81
* 16.81


In [10]:
def prep_data(url):
    
    df = pd.read_parquet(url)

    # Calculate duration
    df['duration'] = (df.tpep_dropoff_datetime - df.tpep_pickup_datetime).dt.total_seconds() / 60
    
    # Filter records
    df_filt = df[(df['duration'] >= 1) & (df['duration'] <= 60) ]

    features = ['PULocationID', 'DOLocationID']

    # Convert the catergoricals to strings (they are integers originally).
    dicts = df_filt[features].astype(str).to_dict(orient='records')

    # Get the duration in seconds
    y_train = df_filt['duration'].values
    
    return dicts, y_train


In [11]:
X_val_dicts, y_val = prep_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

X_val = dv.transform(X_val_dicts)
rows, cols = X_val.shape
    
print(f"The dimensionality of the feature matrix is: {cols}")
    

The dimensionality of the feature matrix is: 515


In [13]:
y_pred_val = lr.predict(X_val)

mse = mean_squared_error(y_val, y_pred_val)
rmse = np.sqrt(mse)

print(f"RMSE on validation is: {rmse}")

RMSE on validation is: 7.81181211389241


>RMSE on validation is: 7.81181211389241


## Submit the results

* Submit your results here: https://courses.datatalks.club/mlops-zoomcamp-2024/homework/hw1
* If your answer doesn't match options exactly, select the closest one
